In [64]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from glob import glob
import os
import navfeatdb.db.pytables as tbdb
import dask.multiprocessing
import h5py
import tables as tb

In [2]:
db_dir = '/data/osip/db/athens/'
out_db = '/data/osip/db/athens_reduced.h5'
feat_per_tile = 10000

In [3]:
slices = sorted(glob(db_dir + '/*.h5'))

In [4]:
# # Add db_idx and file name to each file
# for slice in slices:
#     df = pd.read_hdf(slice, '/db/landmarks')
#     idx_df = pd.DataFrame({'file_idx': df.index, 'filename': slice})
#     idx_df.to_hdf(slice, key='/db/idx', format='table')   

In [5]:
# Load Features
ddf = dd.read_hdf(db_dir + '/*.h5', key='/db/landmarks', lock=False)
idx_df = dd.read_hdf(db_dir + '/*.h5', key='/db/idx', lock=False)
ttdf = dd.concat([ddf, idx_df], axis=1)

/data/anaconda3/lib/python3.5/site-packages/dask/dataframe/multi.py:592: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indexes of each dataframes are aligned
.This assumption is not generally safe.
  warn("Concatenating dataframes with unknown divisions.\n"


In [6]:
ttdf['pair_id'] = ttdf.apply(lambda x: int(tbdb.elegant_pair_xy(x.x, x.y)), axis=1, meta=np.int)

In [7]:
# zz = ttdf.groupby('pair_id').apply(lambda x: x.nlargest(feat_per_tile, columns='response')).reset_index(drop=True)
from dask.diagnostics import ProgressBar
with ProgressBar():
    zz = ttdf.compute(get=dask.multiprocessing.get)

[########################################] | 100% Completed | 56min 14.8s


In [8]:
# zz = ttdf.groupby('pair_id').apply(lambda x: x.nlargest(feat_per_tile, columns='response')).reset_index(drop=True)
# zz.to_hdf(out_db, key='/db/landmarks', format='table', get=dask.multiprocessing.get)

,pix_x,pix_y,angle,response,size,octave,layer,scale,lon,lat,height,x,y,z,file_idx,filename,pair_id
0,4469.292969,4617.707031,114.362305,194.205078,27.620825,2.0,0.0,0.25000,-82.106448,39.551657,201.939999,8910,12458,15,0,/data/osip/db/athens/slice_10.h5,155210674
1,4654.056152,4109.899414,221.920807,184.776672,12.836065,0.0,0.0,1.00000,-82.105785,39.553049,183.939999,8910,12458,15,1,/data/osip/db/athens/slice_10.h5,155210674
2,4689.158691,4114.980469,78.554367,178.205734,26.922884,2.0,0.0,0.25000,-82.105661,39.553035,183.939999,8910,12458,15,2,/data/osip/db/athens/slice_10.h5,155210674
3,4920.788574,4081.933105,270.393738,175.583313,12.661625,0.0,0.0,1.00000,-82.104839,39.553123,182.939999,8910,12458,15,3,/data/osip/db/athens/slice_10.h5,155210674
4,4653.952148,4109.128906,224.776077,160.993286,14.336834,1.0,0.0,0.50000,-82.105786,39.553051,183.939999,8910,12458,15,4,/data/osip/db/athens/slice_10.h5,155210674
5,4826.913086,4179.497070,196.623581,159.488464,11.761279,0.0,0.0,1.00000,-82.105173,39.552856,180.939999,8910,12458,15,5,/data/osip/db/athens/slice_10.h5,155210674
6,4679.791016,4100.119141,246.114395,153.188721,26.308697,2.0,0.0,0.25000,-82.105694,39.553076,187.939999,8910,12458,15,6,/data/osip/db/athens/slice_10.h5,155210674
7,1473.684937,4466.920898,218.931427,152.689880,14.294922,0.0,0.0,1.00000,-82.117070,39.552106,187.939999,8909,12458,15,7,/data/osip/db/athens/slice_10.h5,155210673
8,4722.791504,4094.111328,226.617889,151.206421,12.291428,0.0,0.0,1.00000,-82.105541,39.553092,185.939999,8910,12458,15,8,/data/osip/db/athens/slice_10.h5,155210674
9,1126.358032,4421.796387,215.070023,151.097992,11.926935,0.0,0.0,1.00000,-82.118301,39.552234,185.939999,8909,12458,15,9,/data/osip/db/athens/slice_10.h5,155210673


In [15]:
reduced_df = zz.groupby('pair_id').apply(lambda x: x.nlargest(feat_per_tile, columns='response')).reset_index(drop=True)

In [19]:
reduced_df.sort_values(by=['pair_id', 'response'], ascending=False, inplace=True)

reduced_df = reduced_df.drop_duplicates()

reduced_df.reset_index(inplace=True, drop=True)

reduced_df.to_hdf(out_db, key='/db/landmarks', format='table', data_columns=['pair_id'], complib='zlib', complevel=5)

out_hdf5 = h5py.File(out_db + '.descriptors.h5', 'w')
out_desc = out_hdf5.create_dataset('/db/descriptors', dtype=np.uint8, shape=(reduced_df.shape[0], 64), chunks=(500, 64), compression='gzip', compression_opts=5)

In [132]:
groups = reduced_df.groupby('filename')
for key, group in groups:
    print(key)
    group_h5py = tb.open_file(key, 'r')
    group_desc = group_h5py.root.db.descriptors
    out_desc[group.index] = group_desc[group.file_idx.tolist(), :]
    group_h5py.close()

/data/osip/db/athens/slice_10.h5
/data/osip/db/athens/slice_106.h5
/data/osip/db/athens/slice_107.h5
/data/osip/db/athens/slice_108.h5
/data/osip/db/athens/slice_109.h5
/data/osip/db/athens/slice_11.h5
/data/osip/db/athens/slice_110.h5
/data/osip/db/athens/slice_111.h5
/data/osip/db/athens/slice_112.h5
/data/osip/db/athens/slice_113.h5
/data/osip/db/athens/slice_12.h5
/data/osip/db/athens/slice_13.h5
/data/osip/db/athens/slice_139.h5
/data/osip/db/athens/slice_14.h5
/data/osip/db/athens/slice_140.h5
/data/osip/db/athens/slice_141.h5
/data/osip/db/athens/slice_142.h5
/data/osip/db/athens/slice_143.h5
/data/osip/db/athens/slice_144.h5
/data/osip/db/athens/slice_145.h5
/data/osip/db/athens/slice_146.h5
/data/osip/db/athens/slice_172.h5
/data/osip/db/athens/slice_173.h5
/data/osip/db/athens/slice_174.h5
/data/osip/db/athens/slice_175.h5
/data/osip/db/athens/slice_176.h5
/data/osip/db/athens/slice_177.h5
/data/osip/db/athens/slice_178.h5
/data/osip/db/athens/slice_179.h5
/data/osip/db/athen

In [134]:
out_hdf5.close()